# Prep filtered scaffold sets for distributed design

### Imports

In [1]:
%load_ext lab_black
# python internal
from glob import glob
import os
import socket
import sys

# conda/pip
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas

# notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/home/pleung/projects/crispy_shifty/notebooks
running on node: dig94


### Load a dataframe of filtered scaffolds and associated metadata
These scaffolds had AF2 run on them; for their best quality prediction out of the 5 AF2 ptm models they have > 92 plddt and < 1.5 RMSD to design. 

In [2]:
inputs = pd.read_csv(
    f"/mnt/home/pleung/projects/crispy_shifty/scaffolds/all_filtered.csv"
)

### Domesticate the scaffolds by trimming off leading and trailing loops, designing away disulfides and adding metadata to the output pdb.bz2s. 
Will need to refold the resultant no_cys variants to check that they're ok

In [3]:
from pyrosetta.distributed.packed_pose.core import PackedPose
from pyrosetta.distributed import requires_init
from pyrosetta.rosetta.core.pose import Pose
from typing import *


@requires_init
def path_to_pose_or_ppose(
    path: "", cluster_scores=False, pack_result=False
) -> Generator[str, Union[PackedPose, Pose], None]:
    """
    Generate PackedPose objects given an input path to a file on disk to read in.
    Can do pdb, pdb.bz2, pdb.gz or binary silent file formats.
    To use silents, must initialize Rosetta with "-in:file:silent_struct_type binary".
    Does not save scores unless reading in pdb.bz2 and cluster_scores is set to true.
    This function can be distributed (best for single inputs) or run on a host process
    """
    import bz2
    import pyrosetta.distributed.io as io
    from pyrosetta.distributed import cluster

    if ".silent" in path:
        pposes = io.poses_from_silent(path)  # returns a generator
    elif ".bz2" in path:
        with open(path, "rb") as f:  # read bz2 bytestream, decompress and decode
            ppose = io.pose_from_pdbstring(bz2.decompress(f.read()).decode())
        if cluster_scores:  # set scores in pose after unpacking, then repack
            scores = pyrosetta.distributed.cluster.get_scores_dict(path)["scores"]
            pose = io.to_pose(ppose)
            for key, value in scores.items():
                pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, key, str(value))
            ppose = io.to_packed(pose)
        else:
            pass
        pposes = [ppose]
    elif ".pdb" in path:  # should handle pdb.gz as well
        pposes = [io.pose_from_file(path)]
    else:
        raise RuntimeError("Must provide a pdb, pdb.gz, pdb.bz2, or binary silent")
    for ppose in pposes:
        if pack_result:
            yield ppose
        else:
            pose = io.to_pose(ppose)
            yield pose


@requires_init
def remove_terminal_loops(packed_pose_in=None, **kwargs) -> List[PackedPose]:
    """
    Use DSSP and delete region mover to idealize inputs. Add metadata.
    Assumes a monomer. Must provide either a packed_pose_in or "pdb_path" kwarg.
    """

    import pyrosetta
    import pyrosetta.distributed.io as io
    import sys

    # TODO import crispy shifty module
    sys.path.append("/home/pleung/projects/crispy_shifty")
    from protocols.cleaning import path_to_pose_or_ppose

    # generate poses or convert input packed pose into pose
    if packed_pose_in is not None:
        poses = [io.to_pose(packed_pose_in)]
    else:
        poses = path_to_pose_or_ppose(
            path=kwargs["pdb_path"], cluster_scores=False, pack_result=False
        )
    final_pposes = []
    for pose in poses:
        # get secondary structure
        pyrosetta.rosetta.core.scoring.dssp.Dssp(pose).insert_ss_into_pose(pose, True)
        dssp = pose.secstruct()
        # get leading loop from ss
        if dssp[0] == "H":  # in case no leading loop is detected
            py_idx_n_term = 0
        else:  # get beginning index of first occurrence of LH in dssp
            py_idx_n_term = dssp.find("LH")
        # get trailing loop from ss
        if dssp[-1] == "H":  # in case no trailing loop is detected
            py_idx_c_term = -1
        else:  # get ending index of last occurrence of HL in dssp
            py_idx_c_term = dssp.rfind("HL") + 1
        rosetta_idx_n_term, rosetta_idx_c_term = str(py_idx_n_term + 1), str(
            py_idx_c_term + 1
        )
        trimmed_pose = pose.clone()
        # setup trimming mover
        trimmer = pyrosetta.rosetta.protocols.grafting.simple_movers.KeepRegionMover()
        trimmer.start(rosetta_idx_n_term)
        trimmer.end(rosetta_idx_c_term)
        trimmer.apply(trimmed_pose)
        # setup rechain mover to sanitize the trimmed pose
        rechain = pyrosetta.rosetta.protocols.simple_moves.SwitchChainOrderMover()
        rechain.chain_order("1")
        rechain.apply(trimmed_pose)
        trimmed_length = len(trimmed_pose.residues)
        if "metadata" in kwargs:
            metadata = kwargs["metadata"]
        else:
            metadata = {}
        metadata["trimmed_length"] = str(trimmed_length)
        for key, value in metadata.items():
            pyrosetta.rosetta.core.pose.setPoseExtraScore(pose, key, str(value))
        final_pposes.append(io.to_packed(pose))
    return final_pposes


def redesign_disulfides(packed_pose_in=None, **kwargs):
    """
    fixbb fastdesign with beta_nov16 on all cys residues using layerdesign
    """
    
    xml = """
    <ROSETTASCRIPTS>
    <SCOREFXNS>
        <ScoreFunction name="sfxn" weights="beta_nov16" symmetric="0"/>
    </SCOREFXNS>
    <RESIDUE_SELECTORS>
        <ResidueName name="cys" residue_name3="CYS"/>
        <Not name="not_cys" selector="cys"/>
        <Neighborhood name="around_cys" selector="cys"/>
        <Or name="cys_or_around_cys" selectors="cys,around_cys"/>
        <Not name="not_cys_or_around_cys" selector="cys_or_around_cys"/>
        <Layer name="surface" select_core="false" select_boundary="false" select_surface="true" use_sidechain_neighbors="true"/>
        <Layer name="boundary" select_core="false" select_boundary="true" select_surface="false" use_sidechain_neighbors="true"/>
        <Layer name="core" select_core="true" select_boundary="false" select_surface="false" use_sidechain_neighbors="true"/>
        <SecondaryStructure name="sheet" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="E"/>
        <SecondaryStructure name="entire_loop" overlap="0" minH="3" minE="2" include_terminal_loops="true" use_dssp="true" ss="L"/>
        <SecondaryStructure name="entire_helix" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="H"/>
        <And name="helix_cap" selectors="entire_loop">
        <PrimarySequenceNeighborhood lower="1" upper="0" selector="entire_helix"/>
        </And>
        <And name="helix_start" selectors="entire_helix">
        <PrimarySequenceNeighborhood lower="0" upper="1" selector="helix_cap"/>
        </And>
        <And name="helix" selectors="entire_helix">
        <Not selector="helix_start"/>
        </And>
        <And name="loop" selectors="entire_loop">
        <Not selector="helix_cap"/>
        </And>
    </RESIDUE_SELECTORS>
    <TASKOPERATIONS>
    <RestrictAbsentCanonicalAAS name="design" keep_aas="ADEFGHIKLMNPQRSTVWY"/>
    <OperateOnResidueSubset name="pack" selector="not_cys">
    <RestrictToRepackingRLT/>
    </OperateOnResidueSubset>
    <OperateOnResidueSubset name="lock" selector="not_cys_or_around_cys">
    <PreventRepackingRLT/>
    </OperateOnResidueSubset>
    <DesignRestrictions name="layer_design">
    <Action selector_logic="surface AND helix_start"  aas="DEHKPQR"/>
    <Action selector_logic="surface AND helix"        aas="EHKQR"/>
    <Action selector_logic="surface AND sheet"        aas="EHKNQRST"/>
    <Action selector_logic="surface AND loop"         aas="DEGHKNPQRST"/>
    <Action selector_logic="boundary AND helix_start" aas="ADEHIKLNPQRSTVWY"/>
    <Action selector_logic="boundary AND helix"       aas="ADEHIKLNQRSTVWYM"/>
    <Action selector_logic="boundary AND sheet"       aas="DEFHIKLNQRSTVWY"/>
    <Action selector_logic="boundary AND loop"        aas="ADEFGHIKLNPQRSTVWY"/>
    <Action selector_logic="core AND helix_start"     aas="AFILVW"/>
    <Action selector_logic="core AND helix"           aas="AFILVW"/>
    <Action selector_logic="core AND sheet"           aas="FILVWY"/>
    <Action selector_logic="core AND loop"            aas="AFGILPVW"/>
    <Action selector_logic="helix_cap"                aas="DNSTP"/>
    </DesignRestrictions>
    <LimitAromaChi2 name="arochi" />
    </TASKOPERATIONS>
    <FILTERS>
    </FILTERS>
    <MOVERS>
    <FastDesign name="fast_design" scorefxn="sfxn" repeats="2" task_operations="design,pack,lock,layer_design,arochi">
    <MoveMap name="mm" chi="true" bb="false" jump="false" />
    </FastDesign>
    </MOVERS>
    <PROTOCOLS>
      <Add mover="fast_design"/>
    </PROTOCOLS>
    </ROSETTASCRIPTS>




    <ROSETTASCRIPTS>

    <SCOREFXNS>
        <ScoreFunction name="sfxn" weights="beta" symmetric="0" />
    </SCOREFXNS>
    <RESIDUE_SELECTORS>
        <Chain name="chA" chains="A" />
        <ResidueName name="pro_and_gly_positions" residue_name3="PRO,GLY" />
        <ResidueName name="apolar" residue_name3="ALA,CYS,PHE,ILE,LEU,MET,THR,PRO,VAL,TRP,TYR" />
        <Not name="polar" selector="apolar" />

        <Layer name="surface" select_core="false" select_boundary="false" select_surface="true" use_sidechain_neighbors="true"/>
        <Layer name="boundary" select_core="false" select_boundary="true" select_surface="false" use_sidechain_neighbors="true"/>
        <Layer name="core" select_core="true" select_boundary="false" select_surface="false" use_sidechain_neighbors="true"/>
        <SecondaryStructure name="sheet" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="E"/>
        <SecondaryStructure name="entire_loop" overlap="0" minH="3" minE="2" include_terminal_loops="true" use_dssp="true" ss="L"/>
        <SecondaryStructure name="entire_helix" overlap="0" minH="3" minE="2" include_terminal_loops="false" use_dssp="true" ss="H"/>
        <And name="helix_cap" selectors="entire_loop">
            <PrimarySequenceNeighborhood lower="1" upper="0" selector="entire_helix"/>
        </And>
        <And name="helix_start" selectors="entire_helix">
            <PrimarySequenceNeighborhood lower="0" upper="1" selector="helix_cap"/>
        </And>
        <And name="helix" selectors="entire_helix">
            <Not selector="helix_start"/>
        </And>
        <And name="loop" selectors="entire_loop">
            <Not selector="helix_cap"/>
        </And>
    </RESIDUE_SELECTORS>
    <TASKOPERATIONS>        
    </TASKOPERATIONS>
    <MOVERS>
        <SavePoseMover name="save_before_relax" restore_pose="0" reference_name="before_relax"/>
        <FastRelax name="FastRelax" scorefxn="sfxn" repeats="2" batch="false" ramp_down_constraints="false" cartesian="false" bondangle="false" bondlength="false" min_type="dfpmin_armijo_nonmonotone"  >
            <MoveMap name="MM" >
                <Chain number="1" chi="true" bb="true"/>
            </MoveMap>
        </FastRelax>
    </MOVERS>
    <FILTERS>
        <BuriedUnsatHbonds name="buns" use_reporter_behavior="true" report_all_heavy_atom_unsats="true" scorefxn="sfxn" residue_selector="chA" ignore_surface_res="false" print_out_info_to_pdb="true" confidence="0" use_ddG_style="true" burial_cutoff="0.01" dalphaball_sasa="true" probe_radius="1.1" max_hbond_energy="1.5" burial_cutoff_apo="0.2" />
        <ExposedHydrophobics name="exposed_hydrophobics" />
        <Geometry name="geometry" confidence="0" />
        <Holes name="holes_core" threshold="0.0" residue_selector="core" confidence="0"/>
        <Holes name="holes_all" threshold="0.0" confidence="0"/>
        <SSPrediction name="mismatch_probability" confidence="0" cmd="/software/psipred4/runpsipred_single" use_probability="1" mismatch_probability="1" use_svm="1" />
        <PackStat name="packstat" threshold="0" chain="0" repeats="5"/>
        <SSShapeComplementarity name="sc_hlx" verbose="1" loops="0" helices="1" />
        <SSShapeComplementarity name="sc_all" verbose="1" loops="1" helices="1" />
        <ScoreType name="total_score_pose" scorefxn="sfxn" score_type="total_score" threshold="0" confidence="0" />
        <ResidueCount name="count" />
        <CalculatorFilter name="score_per_res" equation="total_score_full / res" threshold="-2.0" confidence="0">
            <Var name="total_score_full" filter="total_score_pose"/>
            <Var name="res" filter="count"/>
        </CalculatorFilter>
        <worst9mer name="wnm_hlx" rmsd_lookup_threshold="0.4" confidence="0" only_helices="true" />
        <worst9mer name="wnm_all" rmsd_lookup_threshold="0.4" confidence="0" />
        <worst9mer name="9mer" rmsd_lookup_threshold="0.4" confidence="0" />
        <Rmsd name="rmsd_mbf" reference_name="before_relax" chains="A" superimpose="1" threshold="5" by_aln="0" confidence="0" />
        <MoveBeforeFilter name="rmsd" mover="FastRelax" filter="rmsd_mbf" confidence="0" />
    </FILTERS>
    <APPLY_TO_POSE>
    </APPLY_TO_POSE>
    <PROTOCOLS>
        <Add filter_name="buns" />
        <Add filter_name="exposed_hydrophobics" />
        <Add filter_name="geometry" />
        <Add filter_name="holes_core"/>
        <Add filter_name="holes_all"/>
        <Add filter_name="mismatch_probability" />
        <Add filter_name="packstat"/>
        <Add filter_name="sc_hlx" />
        <Add filter_name="sc_all" />
        <Add filter_name="score_per_res" />
        <Add filter_name="wnm_hlx"/>
        <Add filter_name="wnm_all"/>
        <Add filter_name="9mer"/>
        <Add mover_name="save_before_relax" />
        <Add filter_name="rmsd" />
    </PROTOCOLS>
    <OUTPUT scorefxn="sfxn" />
    </ROSETTASCRIPTS>
    """
    return


# add_metadata

In [4]:
metadata = dict(inputs.iloc[-1])
metadata_to_keep = [
    "pdb",
    "topo",
    "best_model",
    "best_average_plddts",
    "best_ptm",
    "best_rmsd_to_input",
    "best_average_DAN_plddts",
    "scaffold_type",
]
metadata = {k: v for k, v in metadata.items() if k in metadata_to_keep}
pdb_path = metadata["pdb"]

<IPython.core.display.Javascript object>

In [9]:
import pyrosetta
pyrosetta.init("-corrections::beta_nov16 true")

sys.path.append("/home/pleung/projects/crispy_shifty")
from protocols.cleaning import remove_terminal_loops
tpposes = remove_terminal_loops(None, pdb_path=pdb_path, metadata=metadata)

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release 2021.31+release.c7009b3115c22daa9efe2805d9d1ebba08426a54 2021-08-07T10:04:12] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release r292 2021.31+release.c7009b3 c7009b3115c22daa9efe2805d9d1ebba08426a54 http://www.pyrosetta.org 2021-08-07T10:04:12
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /home/pleung/.conda/envs/crispy/lib/python3.8/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=822715005 seed_offset=0 real_seed=822715005 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=822715005 RG_type=mt1993

<IPython.core.display.Javascript object>